In [1]:
from ltn_imp.automation.knowledge_base import KnowledgeBase
import torch
import pandas as pd

In [2]:
!poetry run poe download-medical-datasets

Poe => mkdir -p examples/medical/datasets
Poe => curl -L -o examples/medical/datasets/pima_indians_imputed.csv https://raw.githubusercontent.com/ChristelSirocchi/hybrid-ML/main/pima_indians_imputed.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33428  100 33428    0     0  48512      0 --:--:-- --:--:-- --:--:-- 48446 0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [3]:
import random
import numpy as np

seed = 1282

# Set the random seed for reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
from sklearn.model_selection import train_test_split

test_data = pd.read_csv('datasets/pima_indians_imputed.csv', index_col=0).astype(float)
test_data["DiabetesPedigreeFunction"] =  test_data["DiabetesPedigreeFunction"] * 100
test_data["Pregnancies"] = test_data["Pregnancies"] * 10

y = test_data.iloc[:, -1]

x_train, x_test = train_test_split(test_data, test_size=0.5, random_state=seed, stratify=y)
y_train = x_train.iloc[:, -1]  # Extract labels from the training split
x_train, x_val = train_test_split(x_train, test_size=0.1, random_state=seed, stratify=y_train)

x_train.to_csv('datasets/train.csv')
x_val.to_csv('datasets/val.csv')
x_test.to_csv('datasets/test.csv')

In [5]:
import torch
from sklearn.metrics import matthews_corrcoef

def predict(model, x):
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Ensure x is a tensor and has the right dtype
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        elif x.dtype != torch.float32:
            x = x.float()
        
        # Forward pass through the model
        probs = model(x)
        
        # Apply binary classification threshold at 0.5
        preds = (probs > 0.5).float()
    return preds

def compute_metrics(model, data_loader):
    correct = 0
    total = 0
    
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0
    
    all_true_labels = []
    all_predicted_labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for data, labels in data_loader:
            # Ensure data and labels are the correct dtype
            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data, dtype=torch.float32)
            elif data.dtype != torch.float32:
                data = data.float()
            
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.float32)
            elif labels.dtype != torch.float32:
                labels = labels.float()
            
            # Get predictions
            preds = predict(model, data)
            
            # Squeeze predictions and labels to remove dimensions of size 1
            predicted_labels = preds.squeeze()
            true_labels = labels.squeeze()

            # Ensure the shapes match before comparison
            if predicted_labels.shape != true_labels.shape:
                true_labels = true_labels.view_as(predicted_labels)
            
            # Collect all predictions and true labels for MCC
            all_true_labels.extend(true_labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

            # Count correct predictions
            correct += (predicted_labels == true_labels).sum().item()
            total += true_labels.size(0)
            
            # Calculate TP, FP, FN, TN
            true_positives += ((predicted_labels == 1) & (true_labels == 1)).sum().item()
            false_positives += ((predicted_labels == 1) & (true_labels == 0)).sum().item()
            false_negatives += ((predicted_labels == 0) & (true_labels == 1)).sum().item()
            true_negatives += ((predicted_labels == 0) & (true_labels == 0)).sum().item()
    
    accuracy = correct / total if total > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    balanced_accuracy = 0.5 * (recall + (true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0))
    mcc = matthews_corrcoef(all_true_labels, all_predicted_labels)
    tnr = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0  # True Negative Rate
    fpr = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0  # False Positive Rate
    fnr = false_negatives / (false_negatives + true_positives) if (false_negatives + true_positives) > 0 else 0  # False Negative Rate
    tpr = recall  # True Positive Rate is the same as recall

    print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}, True Negatives: {true_negatives}, Total: {total}")
    print()
    print(f"A (Accuracy): {accuracy:.4f}")
    print(f"P (Precision): {precision:.4f}")
    print(f"R (Recall): {recall:.4f}")
    print(f"F1 (F1 Score): {f1_score:.4f}")
    print(f"BA (Balanced Accuracy): {balanced_accuracy:.4f}")
    print(f"MCC (Matthews Correlation Coefficient): {mcc:.4f}")
    print(f"TNR (True Negative Rate): {tnr:.4f}")
    print(f"FPR (False Positive Rate): {fpr:.4f}")
    print(f"FNR (False Negative Rate): {fnr:.4f}")
    print(f"TPR (True Positive Rate): {tpr:.4f}")
    model.train()

In [6]:
kb = KnowledgeBase("with_logic.yaml")

Using device: cpu


In [7]:
kb.predicates["Diabetic"]

Sequential(
  (0): Linear(in_features=8, out_features=256, bias=True)
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=256, out_features=128, bias=True)
  (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=128, out_features=64, bias=True)
  (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): LeakyReLU(negative_slope=0.01)
  (9): Linear(in_features=64, out_features=1, bias=True)
  (10): Sigmoid()
)

In [8]:
fine_tune = True

In [9]:
if fine_tune:
    model = kb.predicates["Diabetic"]

    criteria = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

    patience = 5
    min_delta = 0.0
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(50):
        model.train()
        total_loss = 0.0
        num_batches = 0
        
        # Training loop
        for data, labels in kb.loaders[0]:
            optimizer.zero_grad()
            predictions = model(data)
            loss = criteria(predictions, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1

        avg_train_loss = total_loss / num_batches

        model.eval()
        total_val_loss = 0.0
        num_val_batches = 0

        for data, labels in kb.val_loaders[0]:
            with torch.no_grad():
                predictions = model(data)
                val_loss = criteria(predictions, labels)
                total_val_loss += val_loss.item()
                num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches

        # Early stopping logic
        if avg_val_loss + min_delta < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early Stopping at: Epoch {epoch + 1}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
            break

        if epoch % 10 == 0:
            print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

        model.train()

Epoch 1, Train Loss: 0.647548, Val Loss: 0.691581
Epoch 11, Train Loss: 0.458084, Val Loss: 0.586451
Early Stopping at: Epoch 13, Train Loss: 0.439744, Val Loss: 0.581813


In [10]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 96, False Positives: 30, False Negatives: 24, True Negatives: 195, Total: 345

A (Accuracy): 0.8435
P (Precision): 0.7619
R (Recall): 0.8000
F1 (F1 Score): 0.7805
BA (Balanced Accuracy): 0.8333
MCC (Matthews Correlation Coefficient): 0.6595
TNR (True Negative Rate): 0.8667
FPR (False Positive Rate): 0.1333
FNR (False Negative Rate): 0.2000
TPR (True Positive Rate): 0.8000


In [11]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 91, False Positives: 54, False Negatives: 43, True Negatives: 196, Total: 384

A (Accuracy): 0.7474
P (Precision): 0.6276
R (Recall): 0.6791
F1 (F1 Score): 0.6523
BA (Balanced Accuracy): 0.7316
MCC (Matthews Correlation Coefficient): 0.4553
TNR (True Negative Rate): 0.7840
FPR (False Positive Rate): 0.2160
FNR (False Negative Rate): 0.3209
TPR (True Positive Rate): 0.6791


In [12]:
if fine_tune:
    kb.optimize(num_epochs=100, log_steps=10, lr=0.000001, early_stopping=True, patience=5)
else:
    kb.optimize(num_epochs=150, log_steps=10, lr=0.00001, early_stopping=False, patience=5)

["∀ ['person']. (((y == diabetes) -> Diabetic(person)))", "∀ ['person']. (((y == healthy) -> ~(Diabetic(person))))", "∀ ['person']. ((((person[BMI] > 29) & (person[Glucose] > 125)) -> Diabetic(person)))", "∀ ['person']. ((((person[BMI] < 26) & (person[Glucose] < 101)) -> ~(Diabetic(person))))"]
Rule Outputs:  [tensor(0.7858, grad_fn=<RsubBackward1>), tensor(0.7350, grad_fn=<RsubBackward1>), tensor(0.7607, grad_fn=<RsubBackward1>), tensor(0.8887, grad_fn=<RsubBackward1>)]
Epoch 1/100, Train Loss: 0.21547681093215942, Validation Loss: 0.27148348093032837

["∀ ['person']. (((y == diabetes) -> Diabetic(person)))", "∀ ['person']. (((y == healthy) -> ~(Diabetic(person))))", "∀ ['person']. ((((person[BMI] > 29) & (person[Glucose] > 125)) -> Diabetic(person)))", "∀ ['person']. ((((person[BMI] < 26) & (person[Glucose] < 101)) -> ~(Diabetic(person))))"]
Rule Outputs:  [tensor(0.7946, grad_fn=<RsubBackward1>), tensor(0.7391, grad_fn=<RsubBackward1>), tensor(0.7650, grad_fn=<RsubBackward1>), tenso

In [13]:
print(kb.parameters())

[Parameter containing:
tensor([[ 0.2610, -0.3188,  0.3188,  ...,  0.0629, -0.1324,  0.0576],
        [ 0.1977, -0.1469,  0.1840,  ..., -0.1251, -0.3041, -0.0670],
        [ 0.0978, -0.0491,  0.1389,  ...,  0.2766, -0.3446, -0.0143],
        ...,
        [ 0.2690, -0.1352,  0.2424,  ..., -0.1739,  0.2000,  0.0111],
        [ 0.1253,  0.1610, -0.0482,  ...,  0.1862,  0.2549,  0.2747],
        [ 0.3172,  0.2625,  0.0434,  ..., -0.2252, -0.0135, -0.0935]],
       requires_grad=True), Parameter containing:
tensor([ 1.9005e-01,  2.2431e-01, -3.3059e-01, -2.4056e-01,  2.5840e-01,
        -8.8289e-02,  2.9633e-01,  1.5481e-01,  9.7468e-02,  1.8016e-01,
         1.3124e-01, -3.1086e-01, -1.1611e-01, -1.3264e-02, -9.1153e-02,
        -7.0132e-02,  2.9959e-01, -1.7538e-02, -8.4612e-02,  2.3891e-01,
         3.9871e-02, -2.2504e-01, -2.1186e-01,  1.4001e-01,  3.1702e-01,
         1.7680e-01,  2.6448e-01,  1.3681e-01,  2.8491e-01, -1.4341e-01,
         3.2376e-02,  4.6694e-03,  7.1602e-02,  5.6533e

In [14]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 89, False Positives: 32, False Negatives: 31, True Negatives: 193, Total: 345

A (Accuracy): 0.8174
P (Precision): 0.7355
R (Recall): 0.7417
F1 (F1 Score): 0.7386
BA (Balanced Accuracy): 0.7997
MCC (Matthews Correlation Coefficient): 0.5983
TNR (True Negative Rate): 0.8578
FPR (False Positive Rate): 0.1422
FNR (False Negative Rate): 0.2583
TPR (True Positive Rate): 0.7417


In [15]:
compute_metrics(kb.predicates["Diabetic"], kb.test_loaders[0])

True Positives: 85, False Positives: 57, False Negatives: 49, True Negatives: 193, Total: 384

A (Accuracy): 0.7240
P (Precision): 0.5986
R (Recall): 0.6343
F1 (F1 Score): 0.6159
BA (Balanced Accuracy): 0.7032
MCC (Matthews Correlation Coefficient): 0.4012
TNR (True Negative Rate): 0.7720
FPR (False Positive Rate): 0.2280
FNR (False Negative Rate): 0.3657
TPR (True Positive Rate): 0.6343
